In [1]:
import csv
import pandas as pd

In [2]:
##########################################################################################
processDate = "2016-09-01"

def removeNonASCIICharacters(textString): 
    return "".join(i for i in textString if ord(i)<128)

df = pd.read_csv(r'C:\Users\goldm\Capstone\data\articles.csv')
df.drop_duplicates('content')
df = df[~df['content'].isnull()]
df=df[df['content'].str.len()>=200]

targetString="(Want to get this briefing by email?"
df['NYT summary']=df['content'].map(lambda d: d[:len(targetString)]==targetString)
df=df[df['NYT summary']==False]

# The following removes a warning that appears in many of the Atlantic articles.
# Since it is commonly at the beginning, it brings a lot of noise to the search for similar articles
# And subsequently to the assessment of sentiment
targetString="For us to continue writing great stories, we need to display ads.             Please select the extension that is blocking ads.     Please follow the steps below"
df['content']=df['content'].str.replace(targetString,'')

# This is also for some Atlantic articles for the same reasons as above
targetString="This article is part of a feature we also send out via email as The Atlantic Daily, a newsletter with stories, ideas, and images from The Atlantic, written specially for subscribers. To sign up, please enter your email address in the field provided here."
df=df[df['content'].str.contains(targetString)==False]

# This is also for some Atlantic articles for the same reasons as above
targetString="This article is part of a feature we also send out via email as Politics  Policy Daily, a daily roundup of events and ideas in American politics written specially for newsletter subscribers. To sign up, please enter your email address in the field provided here."
df=df[df['content'].str.contains(targetString)==False]

# More Atlantic-specific removals (for daily summaries with multiple stories contained)
df=df[df['content'].str.contains("To sign up, please enter your email address in the field")==False]

# Remove daily CNN summary
targetString="CNN Student News"
df=df[df['content'].str.contains(targetString)==False]

print("\nArticle counts by publisher:")
print(df['publication'].value_counts())

print("\nArticle counts by date:")
print(df['date'].value_counts())

# Restrict to articles on the provided input date.
# This date is considered mandatory for topic clustering but is not required for sentiment
# since sentiment only processes a specified list of articles.
# For topic clustering it is essential to have the date as it is
# enormously significant in article matching.
# if processDate!=None:
#     df=df[df['date']==processDate]
# df.reset_index(inplace=True, drop=True)

# Remove non-ASCII characters
df['content no nonascii']=df['content'].map(lambda x: removeNonASCIICharacters(x))

print("\nFinal dataset:\n\nDate:",processDate,"\n")
print(df['publication'].value_counts())


Article counts by publisher:
Breitbart           104
NY Post              61
CNN                  57
Reuters              56
NPR                  54
NY Times             50
Washington Post      50
Buzzfeed News        48
Atlantic             48
Business Insider     41
Guardian             35
National Review      32
Fox News             28
Name: publication, dtype: int64

Article counts by date:
2016-12-02    362
2016-09-01    302
Name: date, dtype: int64

Final dataset:

Date: 2016-09-01 

Breitbart           104
NY Post              61
CNN                  57
Reuters              56
NPR                  54
NY Times             50
Washington Post      50
Buzzfeed News        48
Atlantic             48
Business Insider     41
Guardian             35
National Review      32
Fox News             28
Name: publication, dtype: int64


In [25]:
article_df = df

In [3]:
def loadStopWords(stopWordsFileName):
    stop_words=[]
    f=open(stopWordsFileName,'r')
    for l in f.readlines():
        stop_words.append(l.replace('\n', ''))
    return stop_words
stop_words = loadStopWords(runParams['stop_words_file'][0])
#NOTE: alternative to importing a flat file of stop words is to just import stop words from the various different libraries. 
# from nltk.corpus import stopwords
# stop_words = set(stopwords.words('english'))

NameError: name 'runParams' is not defined

In [36]:
import argparse
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import ParameterGrid
import csv

runParams={'tfidf_maxdf':      [0.5],
           'input_file':       ['./data/articles.csv'],
           'story_threshold':  [0.26],
           'process_date':     ['2016-09-01'],
           'parts_of_speech':  [['PROPER', 'VERB']],
           'lemma_conversion': [False],
           'ngram_max':        [3],
           'tfidf_binary':     [False],
           'tfidf_norm':       ['l2'],
           'nlp_library':      ['nltk'],
           'max_length':       [50],
           'stop_words_file':  ['./data/stopWords.txt'],
           'tfidf_mindf':      [2],
           'display_graph':    [True],
           'article_stats':    [False]}
# Use parameter grid even if there is only one set of parameters
parameterGrid = ParameterGrid(runParams)

partsOfSpeech=[]
pos_nlp_mapping = {}
pos_nlp_mapping['nltk']={'VERB':['VB','VBD','VBG','VBN','VBP','VBZ'],'PROPER':['NNP','NNPS'],'COMMON':['NN','NNS']}

for pos in runParams['parts_of_speech'][0]:
    partsOfSpeech.append(pos_nlp_mapping['nltk'][pos])
partsOfSpeech=[item for sublist in partsOfSpeech for item in sublist]
print(partsOfSpeech)

import nltk as nl
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer=WordNetLemmatizer()

stringToConvert = article_df['content']
partsOfSpeech = partsOfSpeech
stop_words = stop_words

['NNP', 'NNPS', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']


# initializing the "story map" - aka the ground truth used to measure the accuracy of our model

In [23]:
import nltk as nl
from nltk import word_tokenize, sent_tokenize

def stringNLTKProcess(nl, stringToConvert,partsOfSpeech,stop_words,maxWords=None,lemmatizer=None):
    #parses the paragraph into sentences
    sentences = nl.sent_tokenize(stringToConvert)
    str = []
    for sentence in sentences:
        wordString=[]
        for word, pos in nl.pos_tag(nl.word_tokenize(sentence)):
            # The following condition avoids any POS which corresponds to punctuation (and takes all others
            if partsOfSpeech == None:
                if pos[0]>='A' and pos[0]<='Z':
                    wordString.append(word)
            elif pos in partsOfSpeech:
                wordString.append(word)
        for wrd in wordString:
            #converts all string characters into lowercase elements
            wrdlower=wrd.lower()
            if wrdlower not in stop_words and wrdlower!="'s'":
                if maxWords==None or len(str)<maxWords:
                    if lemmatizer==None:
                        str.append(wrdlower)
                    else:
                        str.append(lemmatizer.lemmatize(wrd.lower(), pos='v'))
            if maxWords!=None and len(str)==maxWords:
                return ' '.join(str)
    return ' '.join(str)

def removeSpacesAndPunctuation(textString): 
    return "".join(i for i in textString if (ord(i)>=48 and ord(i)<=57) or (ord(i)>=97 and ord(i)<=122))

In [89]:
def setupStoryMapAndReportList(args=None, reportArticleList=None,storyMapFileName=None):
    # Story Map is used in fitting if grid search is applied (As ground truth)
    # It is also used in graph if no threshold provided (to determine colours, not to determine location)
    # Report Article List is used at the end to create a report with, for each
    # article in the list, the set of articles within tolerance, and the key words for each
    if args==None:
        articleList=reportArticleList
        fileName=storyMapFileName
    else:
        articleList=args['article_id_list']
        fileName=args['story_map_validation']
    
    reportArticleList=articleList
    if fileName!=None:
        storyMap=readStoryMapFromFile(fileName)
        if reportArticleList==None:
            reportArticleList=[]
            for story, articleList in storyMap.items():
                reportArticleList.append(articleList[0])
    else:
        storyMap=None
    return storyMap,reportArticleList

def readStoryMapFromFile(filename):
    return readDictFromCsvFile(filename,'StoryMap')

def readGridParameterRangeFromFile(filename):
    return readDictFromCsvFile(filename, 'GridParameters')

def readDictFromCsvFile(filename,schema):
    gridParamDict={} 
    with open(filename,'r') as f:
        for row in f:
            row=row[:-1] # Exclude the carriage return
            row=row.split(',')
            key=row[0]
            vals=row[1:]
            
            if schema=='GridParameters':
                if key in ['story_threshold','tfidf_maxdf']:
                    finalVals=list(float(n) for n in vals)
                elif key in ['ngram_max','tfidf_mindf','max_length']:
                    finalVals=list(int(n) for n in vals)
                elif key in ['lemma_conversion','tfidf_binary']:
                    finalVals = list(str2bool(n) for n in vals)
                elif key in ['parts_of_speech']:
                    listlist=[]
                    for v in vals:
                        listlist.append(v_split('+'))
                    finalVals = listlist
                elif key in ['tfidf_norm','nlp_library']:
                    finalVals=vals
                else:
                    print(key)
                    print("KEY ERROR")
                    return
            elif schema == 'StoryMap':
                finalVals = list(int(n) for n in vals)
            else:
                print(schema)
                print('SCHEMA ERROR')
                return
            
            gridParamDict[key]=finalVals
    return gridParamDict

In [90]:
storyMap,reportArticleList=setupStoryMapAndReportList(storyMapFileName='storyMapForValidation.csv')

In [27]:
article_df['input to vectorizer'] = article_df['content no nonascii'].map(lambda x: stringNLTKProcess(nl, x,partsOfSpeech,stop_words,maxWords=None,lemmatizer=None))

In [49]:
def preprocessAndVectorize(articleDataFrame, args, pos_nlp_mapping, nlp,nl, wordnet_lemmatizer,stop_words):
    from sklearn.feature_extraction.text import TfidfVectorizer
    vectorizer = TfidfVectorizer(analyzer='word',
                                ngram_range=(1,args['ngram_max'][0]),
                                lowercase=True,
                                binary=args['tfidf_binary'][0],
                                **optArgsForVectorizer)
    tfidVectors=vectorizer.fit_transform(articleDataFrame['input to vectorizer'])
    terms=vectorizer.get_feature_names()
    return tfidVectors, terms

In [31]:
def initialiseAllNonZeroCoords(tfidVectors):
    #This function just exists isnce it seems to be expensive and I'd rather not call it multiple times
    #Hence it is intended to be called outside of loops in order to simplify the row specific processing
    values=[]
    nzc=zip(*tfidVectors.nonzero())
    
    #In Python 3 the zip can only be iterated through one time before it is automatically realeased
    ## So need to copy the results otherwise the main loop below will no longer work
    pointList=[]
    for i,j in nzc:
        pointList.append([i,j])
        
    for row in range(tfidVectors.shape[0]):
        rowList=[]
        for i,j in pointList:
            if row==i:
                rowList.append(j)
        values.append(rowList)
        
    return values

In [14]:
def scoreCurrentParamGuess(tfidVectors,storyMap,articleDataFrame,threshold,printErrors=False):
    #Work with distances relative to first item in each cluster - even though this is clearly arbitrary since that
    #point could be an outlier in the cluster and hence might cause some problems.
    #But I have to start somewhere - and can refine it later if needed.
    
    nonZeroCoords=initialiseAllNonZeroCoords(tfidVectors)
    score=0
    outGood=0
    outBad=0
    inGood=0
    inBad=0
    for story, storyArticles in storyMap.items():
        leadArticleIndex=articleDataFrame[articleDataFrame['id']==storyArticles[0]].index[0]
        #Compute score of all articles in corpus relative to the first article in the story ( dot product)
        #Then count through the list relative to the threshold (add one for a good result, subtract one for a bad result)
        scores=productRelatednessScores(tfidfVectors,nonZeroCoords,leadArticleIndex)
        rankedIndices=np.argsort(scores)
        foundRelatedArticles=[]
        # The sorting here is not strictly required,ubt i could use it so that once the threshold is passed
        # in the loop, then i infer the remaining results
        for article in reversed(rankedIndices):
            thisArticleIndex=articleDataFrame['id'][article]
            if thisArticleIndex in storyArticles:
                if scores[article]>=threshold:
                    score+=1
                    inGood+=1
                else:
                    score-=1
                    inBad+=1
                    if printErrors:
                        print("ERROR:",thisArticleIndex,"should be in", story)
            else: # article not supposed to be in range
                if scores[article]<=threshold:
                    score+=1
                    outGood+=1
                else:
                    score-=1
                    outBad+=1
                    if printErrors:
                        print("ERROR:", thisArticleIndex,"should NOT be in", story)
        scoreDict={'score':score, 'inGood': inGood, 'inBad': inBad, "outGood": outGood, 'outBad': outBad}
        return scoreDict

In [99]:
article_df[article_df['id']==storyArticles[0]].index[0]

483

In [93]:
for story, storyArticles in storyMap.items():
    print(story, storyArticles)

Trump meeting [151832, 110126, 172078, 48306, 57365, 190512, 26536, 71335, 21499, 23872, 142033, 110133, 23888, 71336, 57366, 71339]
Brazil impeachment [120639, 80103, 25225, 21502, 57362, 120636, 110141]
Kaepernick [40617, 40543, 39520, 80109, 80101, 47403]
Clinton Guccifer [214888, 85803, 47979]
Farage [37252, 37468, 46175]
Anthony Weiner [49480, 110144, 142300, 214934]
SpaceX [38658, 134545, 172095, 214894]
Safe space [21448, 78169, 78171]
Lauer debate [43447, 47078, 138709]
Venezuela [172079, 57375, 190522]
Iran deal [158005, 48823, 57373, 120634]
Penn State [80094, 157527, 214892]
David Brown [172085, 80096, 141886]


In [37]:
def productRelatednessScores(tfidVectors,nonZeroCoords,refRow):
    # instantiates a matrix of zeros with tfidVectors.shape[0] rows corresponding to the number of rows in the tfidVectors array
    scores = [0]*tfidVectors.shape[0]
    for toRow in range(tfidVectors.shape[0]):
        scores[toRow] = sum([(tfidVectors[toRow,w]*tfidVectors[refRow,w]) for w in nonZeroCoords[refRow] if w in nonZeroCoords[toRow]])
    return scores

In [15]:
# Loop across all parameter combinations in grid to determine best set
# If not doing grid search, will just pass through the loop once
bestParamScoreDict={'score':-1000000}
betsParams=parameterGrid[0]

for i,currentParams in enumerate(parameterGrid):
    if len(parameterGrid)>1:
        print("Combination:", i+1, "of", len(parameterGrid))
        print(currentParams)
        
    # Determine tf-idf vectors
    # terms is just used later on if analysis of final results is requested
    
    tfidVectors, terms=preprocessAndVectorize(articleDataFrame,
                                             currentParams,
                                             pos_nlp_mapping,
                                             nlp,
                                             nl,
                                             wordnet_lemmatizer,
                                             stop_words)
    
    # Computes scores if threshold provided (meaning as part of grid search)
    if 'story_threshold' in currentParams and currentParams['story_threshold']!=None:
        scoreDict=scoreCurrentParamGuess(tfidVectors,storyMap,articleDataFrame,currentParams['story_threshold'])
        print(scoreDict)
        
        #Update best so far
        if scoreDict['score']>=bestParamScoreDict['score']:
            if len(parameterGrid)>1:
                print(i+1,"is the best so far!")
            bestParams = currentParams
            bestParamScoreDict = scoreDict
    # End grid/parameter loop

NameError: name 'parameterGrid' is not defined

NameError: name 'nlp' is not defined

# Testing scoring of TF-IDF vectors

In [53]:
#get the TF-IDF vector from article_df
optArgsForVectorizer = {}
tfidfVectors, terms = preprocessAndVectorize(article_df, runParams, pos_nlp_mapping,None,nl,wordnet_lemmatizer,stop_words,**optArgsForVectorizer)

In [54]:
nonZeroCoords=initialiseAllNonZeroCoords(tfidfVectors)

In [82]:
len(nonZeroCoords[2])

404

In [79]:
nzc=list(zip(*tfidfVectors.nonzero()))
nzc[250194]

(663, 70425)

In [70]:
tfidfVectors.shape

(664, 188293)

In [100]:
for story, storyArticles in storyMap.items():
    leadArticleIndex=article_df[article_df['id']==storyArticles[0]].index[0]
    print(leadArticleIndex)

Trump meeting [151832, 110126, 172078, 48306, 57365, 190512, 26536, 71335, 21499, 23872, 142033, 110133, 23888, 71336, 57366, 71339]
Brazil impeachment [120639, 80103, 25225, 21502, 57362, 120636, 110141]
Kaepernick [40617, 40543, 39520, 80109, 80101, 47403]
Clinton Guccifer [214888, 85803, 47979]
Farage [37252, 37468, 46175]
Anthony Weiner [49480, 110144, 142300, 214934]
SpaceX [38658, 134545, 172095, 214894]
Safe space [21448, 78169, 78171]
Lauer debate [43447, 47078, 138709]
Venezuela [172079, 57375, 190522]
Iran deal [158005, 48823, 57373, 120634]
Penn State [80094, 157527, 214892]
David Brown [172085, 80096, 141886]


In [105]:
for story, storyArticles in storyMap.items():
    leadArticleIndex=article_df[article_df['id']==storyArticles[0]].index[0]
    print(leadArticleIndex)

483
395
94
634
68
151
75
4
111
514
510
262
520


In [102]:
#in order to see the scores, we need to first get the reference rows given by the story map
for story, storyArticles in storyMap.items():
        leadArticleIndex=article_df[article_df['id']==storyArticles[0]].index[0]
        #Compute score of all articles in corpus relative to the first article in the story ( dot product)
        #Then count through the list relative to the threshold (add one for a good result, subtract one for a bad result)
        scores=productRelatednessScores(tfidfVectors,nonZeroCoords,leadArticleIndex)
        
        

In [160]:
scores = productRelatednessScores(tfidfVectors,nonZeroCoords,483)
scores[0]
type(scores)

list

In [162]:
scores_df = pd.DataFrame(scores)
scores_df.head()

,0
0,0.001967
1,0.004660
2,0.000832
3,0.001971
4,0.000831


In [130]:
#returns the index of each row for the values sorted least to greatest
rankedIndices=np.argsort(scores)
rankedIndices
print(rankedIndices[-3])
print(article_df['id'][203])
print(scores[483],scores[203], scores[560])


560
59438
1.0 0.1073909348712558 0.09292303950260095


In [159]:
#for the first article, article 483 of the trump meeting, lets get all the articles which were matching it (in the same cluster)
score=0
outGood=0
outBad=0
inGood=0
inBad=0
threshold=0.25
printErrors=None

scores = productRelatednessScores(tfidfVectors,nonZeroCoords,483)
rankedIndices=np.argsort(scores)
for article in reversed(rankedIndices):
    try:
        thisArticleIndex=article_df['id'][article]
        if thisArticleIndex in storyArticles:
            if scores[article]>=threshold:
                score+=1
                inGood+=1
            else:
                score-=1
                inBad+=1
                if printErrors:
                    print("ERROR:",thisArticleIndex,"should be in", story)
        else: # article not supposed to be in range
            if scores[article]<=threshold:
                score+=1
                outGood+=1
            else:
                score-=1
                outBad+=1
                if printErrors:
                    print("ERROR:", thisArticleIndex,"should NOT be in", story)
    except:
        pass
scoreDict={'score':score, 'inGood': inGood, 'inBad': inBad, "outGood": outGood, 'outBad': outBad}
scoreDict

{'score': 647, 'inGood': 0, 'inBad': 3, 'outGood': 651, 'outBad': 1}

In [153]:
article_df['id'][21]

22197

In [156]:
article_df['id'][5:30]

5     21454
6     21455
7     21474
8     21485
9     21489
10    21499
11    21500
12    21502
13    21504
14    21506
15    21508
16    21509
17    21510
18    21511
19    21512
21    22197
22    22464
23    22505
24    22573
25    22578
26    22598
27    22613
28    22918
29    23315
30    23346
Name: id, dtype: int64

In [8]:
with open(r'C:\Users\goldm\Capstone\storyMapForValidation.csv', 'r') as f:
    for row in f:
        row=row[:-1] # Exclude the carriage return
        row=row.split(",")
        key=row[0]
        vals=row[1:]
        print(row)

['Trump meeting', '151832', '110126', '172078', '48306', '57365', '190512', '26536', '71335', '21499', '23872', '142033', '110133', '23888', '71336', '57366', '71339']
['Brazil impeachment', '120639', '80103', '25225', '21502', '57362', '120636', '110141', '', '', '', '', '', '', '', '', '']
['Kaepernick', '40617', '40543', '39520', '80109', '80101', '47403', '', '', '', '', '', '', '', '', '', '']
['Clinton Guccifer', '214888', '85803', '47979', '', '', '', '', '', '', '', '', '', '', '', '', '']
['Farage', '37252', '37468', '46175', '', '', '', '', '', '', '', '', '', '', '', '', '']
['Anthony Weiner', '49480', '110144', '142300', '214934', '', '', '', '', '', '', '', '', '', '', '', '']
['SpaceX', '38658', '134545', '172095', '214894', '', '', '', '', '', '', '', '', '', '', '', '']
['Safe space', '21448', '78169', '78171', '', '', '', '', '', '', '', '', '', '', '', '', '']
['Lauer debate', '43447', '47078', '138709', '', '', '', '', '', '', '', '', '', '', '', '', '']
['Venezuela'

In [9]:
print(vals)

['172085', '80096', '141886', '', '', '', '', '', '', '', '', '', '', '', '', '']


In [13]:
[int(n) for n in vals if n!='']

[172085, 80096, 141886]

In [40]:
df = pd.read_csv(r'C:\Users\goldm\Capstone\data\articles_expanded_7.10.21.csv')
df.drop_duplicates('content')
df = df[~df['content'].isnull()]
df=df[df['content'].str.len()>=200]
list_cols = df.columns
list_cols[9:12]
df.drop(index=[11:], inplace=True)
df.columns

SyntaxError: invalid syntax (<ipython-input-40-d38fb8e5e0bb>, line 7)

In [41]:
filename= r'C:\Users\goldm\Capstone\data\articles_expanded_7.10.21.csv'
df = pd.read_csv(filename)
df.drop_duplicates('content')
df = df[~df['content'].isnull()]
df=df[df['content'].str.len()>=200]

targetString="(Want to get this briefing by email?"
df['NYT summary']=df['content'].map(lambda d: d[:len(targetString)]==targetString)
df=df[df['NYT summary']==False]

# The following removes a warning that appears in many of the Atlantic articles.
# Since it is commonly at the beginning, it brings a lot of noise to the search for similar articles
# And subsequently to the assessment of sentiment
targetString="For us to continue writing great stories, we need to display ads.             Please select the extension that is blocking ads.     Please follow the steps below"
df['content']=df['content'].str.replace(targetString,'')

# This is also for some Atlantic articles for the same reasons as above
targetString="This article is part of a feature we also send out via email as The Atlantic Daily, a newsletter with stories, ideas, and images from The Atlantic, written specially for subscribers. To sign up, please enter your email address in the field provided here."
df=df[df['content'].str.contains(targetString)==False]

# This is also for some Atlantic articles for the same reasons as above
targetString="This article is part of a feature we also send out via email as Politics  Policy Daily, a daily roundup of events and ideas in American politics written specially for newsletter subscribers. To sign up, please enter your email address in the field provided here."
df=df[df['content'].str.contains(targetString)==False]

# More Atlantic-specific removals (for daily summaries with multiple stories contained)
df=df[df['content'].str.contains("To sign up, please enter your email address in the field")==False]

# Remove daily CNN summary
targetString="CNN Student News"
df=df[df['content'].str.contains(targetString)==False]


print("\nArticle counts by publisher:")
print(df['publication'].value_counts())

print("\nArticle counts by date:")
print(df['date'].value_counts())

#     Restrict to articles on the provided input date.
#     This date is considered mandatory for topic clustering but is not required for sentiment
#     since sentiment only processes a specified list of articles.
#     For topic clustering it is essential to have the date as it is
#     enormously significant in article matching.

if processDate!=None:
    df=df[df['date']==processDate]
df.reset_index(inplace=True, drop=True)

# Remove non-ASCII characters
df['content no nonascii']=df['content'].map(lambda x: removeNonASCIICharacters(x))

print("\nFinal dataset:\n\nDate:",processDate,"\n")
print(df['publication'].value_counts())


Article counts by publisher:
Breitbart         104
NY Post            61
Reuters            60
CNN                58
NPR                54
                 ... 
Newsweek            1
HowStuffWorks       1
@ChinaDailyApp      1
T-Roy               1
POLITICO            1
Name: publication, Length: 63, dtype: int64

Article counts by date:
9/1/2016     434
12/2/2016    349
Name: date, dtype: int64

Final dataset:

Date: 2016-09-01 

Series([], Name: publication, dtype: int64)


In [42]:
print(processDate)

2016-09-01


In [52]:
filename= r'C:\Users\goldm\Capstone\data\articles_expanded_7.10.21.csv'
df = pd.read_csv(filename)
df.drop_duplicates('content')
df = df[~df['content'].isnull()]
df=df[df['content'].str.len()>=200]
targetString="(Want to get this briefing by email?"
df['NYT summary']=df['content'].map(lambda d: d[:len(targetString)]==targetString)
df=df[df['NYT summary']==False]
targetString="For us to continue writing great stories, we need to display ads.             Please select the extension that is blocking ads.     Please follow the steps below"
df['content']=df['content'].str.replace(targetString,'')

# This is also for some Atlantic articles for the same reasons as above
targetString="This article is part of a feature we also send out via email as The Atlantic Daily, a newsletter with stories, ideas, and images from The Atlantic, written specially for subscribers. To sign up, please enter your email address in the field provided here."
df=df[df['content'].str.contains(targetString)==False]

# This is also for some Atlantic articles for the same reasons as above
targetString="This article is part of a feature we also send out via email as Politics  Policy Daily, a daily roundup of events and ideas in American politics written specially for newsletter subscribers. To sign up, please enter your email address in the field provided here."
df=df[df['content'].str.contains(targetString)==False]

# More Atlantic-specific removals (for daily summaries with multiple stories contained)
df=df[df['content'].str.contains("To sign up, please enter your email address in the field")==False]

# Remove daily CNN summary
targetString="CNN Student News"
df=df[df['content'].str.contains(targetString)==False]

if processDate!=None:
    df=df[df['date']==processDate]
df.reset_index(inplace=True, drop=True)
    
df.head()

KeyError: False

In [51]:
processDate

'2016-09-01'

In [121]:
df = pd.read_csv(r'./data/articlesv2.csv')

In [122]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 811 entries, 0 to 810
Columns: 362 entries, Unnamed: 0 to Unnamed: 361
dtypes: float64(1), object(361)
memory usage: 2.2+ MB


In [123]:
df.columns

Index(['Unnamed: 0', 'id', 'title', 'publication', 'author', 'date', 'year',
       'month', 'url', 'content',
       ...
       'Unnamed: 352', 'Unnamed: 353', 'Unnamed: 354', 'Unnamed: 355',
       'Unnamed: 356', 'Unnamed: 357', 'Unnamed: 358', 'Unnamed: 359',
       'Unnamed: 360', 'Unnamed: 361'],
      dtype='object', length=362)

In [124]:
df = df[['id', 'title', 'publication', 'author', 'date', 'year',
       'month', 'url', 'content']]

In [129]:
df['date']

Series([], Name: date, dtype: object)

In [128]:
processDate = '2016-09-01'
if processDate!=None:
    df=df[df['date']==processDate]
df.reset_index(inplace=True, drop=True)
df['date']

Series([], Name: date, dtype: object)

In [232]:
orig_articles_df = pd.read_csv(r'./data/articles.csv')
new_articles_df = pd.read_csv(r"C:\Users\goldm\News API\combinedTopics_1-5_import_ready.csv")

In [233]:
orig_articles_df.columns

Index(['Unnamed: 0', 'id', 'title', 'publication', 'author', 'date', 'year',
       'month', 'url', 'content'],
      dtype='object')

In [234]:
new_articles_df.columns

Index(['id', 'time', 'article.text', 'article.author', 'article.pub_date',
       'article.is_article', 'article.url', 'article.title',
       'article.language', 'article.summary', 'article.modified_date',
       'article.site_name', 'article.encoding', 'Topic Classification', 'url'],
      dtype='object')

In [235]:
new_articles_df_shortened = new_articles_df[['id','article.site_name', 'article.pub_date', 'article.text']]
new_articles_df_shortened.columns = ['id','publication','date','content']
new_articles_df_shortened.head()

,id,publication,date,content
0,217417,Global News,7/8/2021 14:23,Haiti in state of emergency after President Jo...
1,217418,@ChinaDailyApp,1/4/2021 11:00,MEXICO CITY - Haitian police have arrested two...
2,217419,Reuters,7/7/2021 13:43,Haiti's President Jovenel Moise speaks during ...
3,217420,The White House,7/7/2021 14:53,We are shocked and saddened to hear of the hor...
4,217421,nypost.com,7/9/2021,"1 of\n\nThe late Jovenel Moise, whom was kill..."


In [236]:
#changing the date to be the exact same as the '2016-09-01'
new_articles_df_shortened['date'] = pd.DataFrame(['2016-09-01' for date in range(0,new_articles_df_shortened.shape[0])])
new_articles_df_shortened['date']

<ipython-input-236-c6fec7e3cf40>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_articles_df_shortened['date'] = pd.DataFrame(['2016-09-01' for date in range(0,new_articles_df_shortened.shape[0])])


0      2016-09-01
1      2016-09-01
2      2016-09-01
3      2016-09-01
4      2016-09-01
          ...    
117    2016-09-01
118    2016-09-01
119    2016-09-01
120    2016-09-01
121    2016-09-01
Name: date, Length: 122, dtype: object

In [237]:
orig_articles_df_shortened = orig_articles_df[['id','publication','date','content']]

In [238]:
combined_df = orig_articles_df_shortened

In [239]:
combined_df = combined_df.append(new_articles_df_shortened)
combined_df.columns

Index(['id', 'publication', 'date', 'content'], dtype='object')

In [240]:
combined_df.head()

,id,publication,date,content
0,20694,NY Times,2016-12-02,"If you've seen one movie apocalypse, you have ..."
1,20875,NY Times,2016-12-02,There was the breadcrumb dropped on Valentine'...
2,20986,NY Times,2016-12-02,CLEVELAND — This was supposed to be Fox New...
3,21413,NY Times,2016-09-01,Another Earth could be circling the star right...
4,21448,NY Times,2016-09-01,The anodyne welcome letter to incoming freshme...


In [241]:
len(combined_df)

795

In [242]:
len(orig_articles_df) + len(new_articles_df)

795

In [243]:
len(new_articles_df)

122

In [244]:
len(orig_articles_df)

673

In [224]:
combined_df[combined_df['id']==217535]

,id,publication,date,content
118,217535,Business Insider,2016-09-01,- Adam Kinzinger told NYT Magazine that he su...


In [225]:
combined_df['date']

0      2016-12-02
1      2016-12-02
2      2016-12-02
3      2016-09-01
4      2016-09-01
          ...    
117    2016-09-01
118    2016-09-01
119    2016-09-01
120    2016-09-01
121    2016-09-01
Name: date, Length: 795, dtype: object

In [245]:
df = pd.DataFrame(combined_df)
processDate = '2016-09-01'
if processDate!=None:
    df=df[df['date']==processDate]
df.reset_index(inplace=True, drop=True)
len(df['date'])

430

In [227]:
len(df['date'])

430

In [246]:
#export the combined articles file to csv
combined_df.to_csv(r'C:\Users\goldm\Capstone\data\articles_combined_formattedv1.csv')

In [254]:
def setupStoryMapAndReportList(args=None, reportArticleList=None,storyMapFileName=None):
    # Story Map is used in fitting if grid search is applied (As ground truth)
    # It is also used in graph if no threshold provided (to determine colours, not to determine location)
    # Report Article List is used at the end to create a report with, for each
    # article in the list, the set of articles within tolerance, and the key words for each
    if args==None:
        articleList=reportArticleList
        fileName=storyMapFileName
    else:
        articleList=args['article_id_list']
        fileName=args['story_map_validation']
    
    reportArticleList=articleList
    if fileName!=None:
        storyMap=readStoryMapFromFile(fileName)
        if reportArticleList==None:
            reportArticleList=[]
            for story, articleList in storyMap.items():
                reportArticleList.append(articleList[0])
    else:
        storyMap=None
    return storyMap,reportArticleList

def readStoryMapFromFile(filename):
    return readDictFromCsvFile(filename,'StoryMap')

def readGridParameterRangeFromFile(filename):
    return readDictFromCsvFile(filename, 'GridParameters')

def readDictFromCsvFile(filename,schema):
    gridParamDict={} 
    with open(filename,'r') as f:
        for row in f:
            row=row[:-1] # Exclude the carriage return
            row=row.split(',')
            key=row[0]
            vals=row[1:]
            
            if schema=='GridParameters':
                if key in ['story_threshold','tfidf_maxdf']:
                    finalVals=list(float(n) for n in vals)
                elif key in ['ngram_max','tfidf_mindf','max_length']:
                    finalVals=list(int(n) for n in vals)
                elif key in ['lemma_conversion','tfidf_binary']:
                    finalVals = list(str2bool(n) for n in vals)
                elif key in ['parts_of_speech']:
                    listlist=[]
                    for v in vals:
                        listlist.append(v_split('+'))
                    finalVals = listlist
                elif key in ['tfidf_norm','nlp_library']:
                    finalVals=vals
                else:
                    print(key)
                    print("KEY ERROR")
                    return
            elif schema == 'StoryMap':
                finalVals = list(int(n) for n in vals if n!='')
            else:
                print(schema)
                print('SCHEMA ERROR')
                return
            
            gridParamDict[key]=finalVals
    return gridParamDict

In [229]:
# getting the information to make the story validation map
story_map_for_validation

NameError: name 'story_map_for_validation' is not defined

In [230]:
combined_df.shape

(795, 4)

In [231]:
combined_df[combined_df['id']== 217490]

,id,publication,date,content
73,217490,Hosted,2016-09-01,"TOPICS\n, , , - Economy ,\n- Coronavirus ,..."


In [248]:
new_articles_df_classifications = new_articles_df[['id','Topic Classification']]

In [250]:
new_articles_df_classifications.to_csv(r'new_article_classificationsv1.csv')

In [255]:
storyMap,reportArticleList=setupStoryMapAndReportList(storyMapFileName='storyMapForValidation_expanded.csv')

In [ ]:
for story, storyArticles in storyMap.items():
    leadArticleIndex=articleDataFrame[articleDataFrame['id']==storyArticles[0]].index[0]
    # Compute score of all articles in corpus relative to first article in story (.product)
    # Then count through list relative to threshold (add one for a good result, subtract one for a bad result)
    scores=productRelatednessScores(tfidfVectors,nonZeroCoords,leadArticleIndex)
    rankedIndices=np.argsort(scores)
    foundRelatedArticles=[]
    # THE SORTING HERE IS NOT STRICTLY REQUIRED, BUT I COULD USE IT SO THAT ONCE THE THRESHOLD IS PASSED
    # IN THE LOOP, THEN I INFER THE REMAINING RESULTS
    for article in reversed(rankedIndices):
        thisArticleIndex=articleDataFrame['id'][article]
        if thisArticleIndex in storyArticles:
            if scores[article]>=threshold: # article IS supposed to be in range
                score+=1
                inGood+=1
                #appending the article and its mapping according to the predictions of our model
                final_mapping_list.append([thisArticleIndex, story, 'TP'])
            else:
                score-=1
                inBad+=1
                final_mapping_list.append([thisArticleIndex, 'No Mapping', 'FN'])
                if printErrors:
                    print("ERROR:",thisArticleIndex,"should be in",story)
        else: # article not supposed to be in range
            if scores[article]<=threshold:
                score+=1
                outGood+=1
                final_mapping_list.append([thisArticleIndex, 'No Mapping', 'TN'])
            else:
                score-=1
                outBad+=1
                final_mapping_list.append([thisArticleIndex, story, 'FP'])
                if printErrors:
                    print("ERROR:",thisArticleIndex,"should NOT be in",story)

#### ---- Richard Modification- adding in code to print out a df of articles in the story map and their respective categor
final_mapping_df = pd.DataFrame(final_mapping_list, columns = ['article', 'cateogry ID', 'FP/FN/TP/TN'])
final_mapping_df